In [110]:
import os
import sys
import pathlib
import matplotlib.pyplot as plt
import math
import random
import json
import numpy as np 
import open3d as o3d
import sklearn.cluster
from sklearn import metrics

# sys.path.append(str(pathlib.Path('task1_lidar_cls.ipynb').parent))
# from libraries.clustering import clustering
from libraries.ground_segmentation import ground_segmentation

file_data = np.fromfile('/home/jidan/Documents/me5413/1_lidar/lidar_data/frame2.pcd.bin', dtype=np.float32)
points = file_data.reshape((-1, 5))[:, :4]
print(np.shape(points))
# Points: (x, y, z, intensity)

GROUND_SEGMENTATION = True
if GROUND_SEGMENTATION:
    ground_cloud,segmented_cloud, index_ground, index_segmented = ground_segmentation(points[:,0:3])

def clustering(points, method):
    if method == 'dbscan':
        db = sklearn.cluster.DBSCAN(eps=2,min_samples=3).fit(points)#eps=1.8, min_samples=20
        labels_db = db.labels_

        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(labels_db)) - (1 if -1 in labels_db else 0)
        n_noise_ = list(labels_db).count(-1)

        print("Estimated number of clusters: %d" % n_clusters_)
        print("Estimated number of noise points: %d" % n_noise_)
        return labels_db

    if method == 'kmeans':
        kmeans = sklearn.cluster.KMeans(n_clusters=70, random_state=0, n_init="auto").fit(points)
        labels_km = kmeans.labels_
        return labels_km

    if method == 'meanshift':
        # The following bandwidth can be automatically detected using
        bandwidth = sklearn.cluster.estimate_bandwidth(points, quantile=0.1, n_samples=500)

        ms = sklearn.cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
        ms.fit(points)
        labels_ms = ms.labels_
        cluster_centers = ms.cluster_centers_

        labels_unique = np.unique(labels_ms)
        n_clusters_ = len(labels_unique)

        print("number of estimated clusters : %d" % n_clusters_)
        return labels_ms

    if method == 'optics':
        clust = sklearn.cluster.OPTICS(min_samples=6)

        # Run the fit
        clust.fit(points)
        labels_op = clust.labels_[clust.ordering_]
        return labels_op

    if method == 'AgglomerativeClustering':
        clustering = sklearn.cluster.AgglomerativeClustering(70).fit(points)
        return clustering.labels_

    if method == 'birch':
        clustring = sklearn.cluster.Birch(n_clusters=None).fit(points)
        labels_brc = clustring.predict(points)
        return labels_brc

#Visualization
def vis(data,label):
    '''
    :param data: n*3 matrix
    :param label: n*1 matrix
    :return: visualization
    '''
    labels=np.asarray(label)
    max_label=labels.max()

    # 颜色
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    pt1 = o3d.geometry.PointCloud()
    pt1.points = o3d.utility.Vector3dVector(data.reshape(-1, 3))
    pt1.colors=o3d.utility.Vector3dVector(colors[:, :3])

    o3d.visualization.draw_geometries([pt1],'part of cloud',width=500,height=500)

(34720, 4)
iters = 44.171657


In [111]:
x = points[:, 0]  # x position of point
y = points[:, 1]  # y position of point
z = points[:, 2]  # z position of point
r = points[:, 3]  # reflectance value of point
d = np.sqrt(x ** 2 + y ** 2)  # Map Distance from sensor
degr = np.degrees(np.arctan(z / d))
 
vals = 'height'
if vals == "height":
    col = z
else:
    col = d

#Clustering
method = 'birch'  #Options: 'dbscan','kmeans','optics','meanshift','AgglomerativeClustering'
labels = clustering(segmented_cloud, method)

#Visualization
vis(segmented_cloud,labels)

#Data saving
points = np.insert(points,3,0,axis = 1)
print(f"shape of points = {np.shape(points)}")

points[index_ground, 4] = 80

points[index_segmented, 4] = labels

vis(points[:,0:3],points[:,4])

points = points.tolist()

with open('lidar_clustering.json', 'w') as f:
    json.dump(points, f)


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f59d73eed30>
Traceback (most recent call last):
  File "/home/jidan/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/jidan/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/jidan/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/jidan/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_m

shape of points = (34720, 5)
